# Data Preparation

## 1 Setup

In [ ]:
from typing import Union

import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler

%matplotlib inline

Funktionen zur Visualisierung der Performance unserer Modelle:

Der R²-Wert (Bestimmtheitsmaß) liegt zwischen 0 und 1, wobei 1 bedeutet, dass das Modell die gesamte Variation der abhängigen Variablen erklärt, und 0 bedeutet, dass das Modell keine über die Vorhersage des Durchschnitts hinausgehende Erklärungskraft hat. Ein R²-Wert von 0,5 zeigt beispielsweise, dass 50% der Schwankungen der tatsächlichen Werte durch das Modell erklärt werden können. Dies bedeutet, dass das Modell einen mäßigen Erfolg bei der Erklärung der beobachteten Schwankungen hat. Es gibt jedoch noch Raum für Verbesserungen, da 50 % der Schwankungen nicht durch das Modell erklärt werden.

Der MAE steht für "Mean Absolute Error" (Durchschnittlicher Absoluter Fehler) und wird verwendet, um die durchschnittliche absolute Abweichung zwischen den tatsächlichen und vorhergesagten Werten zu quantifizieren. Er wird bevorzugt, wenn Ausreißer in den Daten weniger stark gewichtet werden sollen, da der absolute Betrag genommen wird. Je niedriger der MAE, desto besser ist die Modellleistung. Wenn beispielsweise der MAE 0 ist, bedeutet dies, dass das Modell perfekte Vorhersagen getroffen hat. Der MAE ist leicht zu interpretieren, da er angibt, um wie viel Einheiten die durchschnittliche Vorhersage des Modells von den tatsächlichen Werten abweicht.

In [ ]:
def performance_measures(y_true, y_pred) -> tuple:
    """
    Calculate R2 and MAE
    
    Args:
        y_true: array-like
        True values
        y_pred: array-like
        Predicted values
        
    Returns:
        r2: float
        R2 score
        mae: float
        Mean Absolute Error
    """
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"R2: {r2:.2f}")
    print(f"MAE: {mae:.2f}")
    return r2, mae

In [ ]:
def actual_vs_predicted(true_v, pred_v) -> None:
    """
    Plot actual vs. predicted values

    Args:
        true_v: array-like
        True values
        pred_v: array-like
        Predicted values

    Returns:
        None
    """
    data = {"Actual": true_v, "Predicted": pred_v}
    df = pd.DataFrame(data)

    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(10, 6))

    sns.lineplot(data=df, markers=False)

    plt.title("Actual vs. Predicted Values")
    plt.xlabel("Data Points")
    plt.ylabel("Values")

    plt.show()

Diese Klasse wird verwendet um eigene Funktionen in die Pipeline zu integrieren.

In [ ]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    """This class is used to apply custom transformations."""
    def __init__(self, function, config: dict = {}):
        self.function = function
        self.config = config

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        X = self.function(X, **self.config) 
        return X

In [ ]:
def select_numeric(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function selects the numeric columns from a DataFrame.
    
    Args:
        df: DataFrame
        The input DataFrame
    
    Returns:
        DataFrame
        A DataFrame with only numeric columns
    """
    df = df.select_dtypes(include=['number'])
        
    return df

In [ ]:
def drop_features(df: pd.DataFrame, columns: list[str] = []) -> pd.DataFrame:
    """
    Drops specified columns from a DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list of str): List of column names to drop.
    
    Returns:
        pd.DataFrame: A DataFrame with specified columns removed.
    """
    return df.drop(columns=columns, errors='ignore')

## 2 Import der Daten

Wir laden unsere beiden Datensätze in ```DataFrames```.

In [ ]:
train_data = pd.read_csv(f"../data/BikeRentalDaily_train.csv", sep=";", index_col=0)
test_data = pd.read_csv(f"../data/BikeRentalDaily_test.csv", sep=";", index_col=0)

Hier ersetzen wir alle Leerzeichen durch `_`. Relevant für `price reduction`.

In [ ]:
train_data.columns = train_data.columns.str.replace(r"\s", r"_", regex=True)
test_data.columns = test_data.columns.str.replace(r"\s", r"_", regex=True)

Die Summe der beiden Spalten ```casual``` und ```registered``` ergibt den Wert von ```cnt```, diese sind in der Beschreibung des Datensatzes zudem als Labels beschrieben, daher werden sie ebenfalls entfernt.

In [ ]:
features = train_data.columns.drop(["cnt", "casual", "registered"])

## 3 Minimal Preprocessing and Baseline Linear Regression Model

Um unsere Modelle unabhängig von den Testdaten valdieren zu können, splitten wir die ```train_data``` Daten in train und validate.

Dieser Schritt wird vor jeder erneuten Validierung druchgeführt.

Damit wir unser Baseline Modell trainieren können, müssen wir Vorverarbeitungsschritte durchführen. Diese werden in der ```minimal_preprocessing``` Pipeline definiert.

Zum einen entfernen wir alle nicht numerischen Spalten aus den Daten.

Zum anderen werden alle ```NaN``` Werte unter Anwendung des ```SimpleImputer``` durch den jeweiligen Mittlewert aufgefüllt.

In [ ]:
drop_feature = CustomTransformer(function=drop_features, config={"columns": ["dteday"]})

numeric_values = CustomTransformer(function=select_numeric)

minimal_pipeline = Pipeline([
    ("drop_feature", drop_feature),
    ("numeric_values", numeric_values),
    ("imputer", SimpleImputer(strategy="mean"))])

Für die erste Iteration des Preprocessing erzeugen wir eine ```minimal_pipeline``` welche die beiden zuvor erzeugten Pipelines miteinander kombiniert.

In [ ]:
pipeline = Pipeline([("minimal_pipeline", minimal_pipeline)])

Die einzelnen Schritte der erzeugten Pipeline werden nacheinander ausgeführt und das Modell wird auf dem verarbeiteten Datensatz trainiert.

In [ ]:
processed_data = pipeline.fit_transform(train_data[features])

In [ ]:
processed_data_features = pipeline.named_steps['minimal_pipeline'].named_steps['imputer'].get_feature_names_out()

In [ ]:
processed_data = pd.DataFrame(processed_data, columns=processed_data_features, index=train_data.index)

In [ ]:
processed_data.head().T

In [ ]:
processed_data.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(processed_data, train_data["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_01 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_01.fit(X_train, y_train)

Im Anschluss nutzten wir die Validierungsdaten um das Modell damit zu evaluieren.

In [ ]:
y_pred = baseline_model_01.predict(X_validate)

Die beiden zu Beginn definierten Funktionen geben zum einen die ```performance_measures``` und die Darstellung der ```acutual_vs_predicted``` Werte aus.

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Aus dem Diagramm wird ersichtlich, dass unser ```baseline_model``` aufgrund der Ausreißer im Label ```cnt``` noch Schwierigkeiten bei der Vorhersage hat.

In [ ]:
break

## 4 Missing Value Handling

In [ ]:
msno.matrix(train_data)

Wie wir bereits in Task 1 erarbeitet haben, weisen die Spalten `season` und `hum` fehlende Werte auf. 

Im `minimal_preprocessing` werden alle `NaN` Werte mit durch den `SimpleImputer` mit `mean` aufgefüllt.

Nun sollen die fehlenden Werte in der Spalte `season` anhand des vorliegenden Datums in der entsprechenden Zeile aufgefüllt werden. 

`impute_season` bestimmt die Jahreszeit anhand eines Datums im Format `"%d.%m.%y"`. 

Diese liest den Tag des entsprechenden Datums aus und weist diesen der dafür entsprechenden Season zu.

Es gibt vier fest definierte Zeitpunkte für den Beginn jeder Jahreszeit (Frühling, Sommer, Herbst, Winter).

Diese Zeitpunkte sind auf den 20. März, 20. Juni, 20. September und 20. Dezember festgelegt.

Damit wir die entsprechende Season anhand des Datums bestimmen können, müssen wir die Werte der Spalte `dteday` zuvor in `datetime` Objekte umwandeln. 

Dafür nutzen wir die `feature_to_datetime` Funktion, welche der preprocessing Pipeline als CustomTransformer mitgegeben wird.

In [ ]:
def feature_to_datetime(df: pd.DataFrame, target: str, date_format: str = "%d.%m.%Y") -> pd.DataFrame:
    """
    Convert the feature to datetime.
    
    Args:
        df: pd.DataFrame: The DataFrame to be transformed.
        target: str: The feature to be transformed.
        date_format: str: The format of the date.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """
    df[target] = pd.to_datetime(df[target], format=date_format)
    
    return df

In [ ]:
def impute_season(df: pd.DataFrame, target: str, date: str) -> pd.DataFrame:
    """
    Impute season based on target column value.
    
    Args:
        df (pd.DataFrame): DataFrame
        target (str): target column name
    
    Returns:
        pd.DataFrame: DataFrame with season imputed based on target column value
    """
    def get_season(date):
        day_of_year = date.timetuple().tm_yday
        if 80 <= day_of_year < 172:
            return 2
        elif 172 <= day_of_year < 265:
            return 3
        elif 265 <= day_of_year < 355:
            return 4
        else:
            return 1
        
    if date in df.columns:
        df[target] = df[date].apply(get_season)
        
    return df

In der folgenden Zelle definieren wir die einzelnen `CustomTransformer` um die beschriebene Funktionaliät umzusetzten.

In [ ]:
feature_datetime = CustomTransformer(function=feature_to_datetime, config={"target": "dteday"})

season_imputer = CustomTransformer(function=impute_season, config={"target": "season", "date": "dteday"})

missing_value_pipeline = Pipeline([
    ("feature_datetime", feature_datetime),
    ("season_imputer", season_imputer)])

Hier wird die eben definierte Pipeline mit der minimal Pipeline dem ersten Schritt kombiniert.

In [ ]:
pipeline = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("minimal_pipeline", minimal_pipeline)])

Danach holen wir uns den transfromierten Datensatz aus der Pipeline.

In [ ]:
result = pipeline.fit_transform(train_data[features])

In [ ]:
result_features = pipeline.named_steps['minimal_pipeline'].named_steps['imputer'].get_feature_names_out()

Wir entfernen das dteday Feature aus der temp_features liste, weil es durch den numeric_values Transformer aus den transformierten Daten entfernt wurde und wandeln das result in einen Dataframe um.

In [ ]:
result = pd.DataFrame(result, columns=result_features, index=train_data.index)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_01 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_01.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_01.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Anhand der Performace Measures ist zu erkennen, dass sich durch diesen Schritt keine wesentlichen Verbesserungen ergeben haben.

## 5 Data Corrections

Aus der deskriptiven Statistik ist ersichltich, dass das Minimum der Variable `windspeed` bei `-1` liegt. 

Da die Windgeschwindigkeit nicht negativ sein kann, treffen wir die Annahme, dass es sich hierbei um Fehler handeln muss.

Hier würde sich ggf. eine Regression Imputation zum Auffüllen der vier fehlenden Werte anbieten.

Aus der Correlation Matrix sehen wir aber, dass `windspeed` kaum mit anderen Features korreliert. Aus diesem Grund entfernen wir die Werte.

Hierfür nutzen wir folgende Funktion, welche die negativen Werte einer Spalte dropt.

In [ ]:
def drop_rows(df: pd.DataFrame, target: str, thresh: Union[int, float]) -> pd.DataFrame:
    """
    Drop rows based on a threshold.
    
    Args:
        df: pd.DataFrame: The DataFrame to be transformed.
        target: str: The target column.
        threshold: float: The threshold for dropping rows.

    Returns:
        pd.DataFrame: The transformed DataFrame.
    """
    try:
        df = df.loc[df[target] >= thresh]
        print(f"{df.shape[0]} rows remaining")
    except KeyError:
        print(f"{target} not found in DataFrame")
    return df

In [ ]:
def process_weekday(df: pd.DataFrame, target: str, date: str) -> pd.DataFrame:
    """
    Maps weekday based on date column values.
    
    Args:
        df (pd.DataFrame): DataFrame
        date (str): date column name (in datetime64 format)
    
    Returns:
        pd.DataFrame: DataFrame with 'weekday' column mapped based on date column values.
    """
    if date in df.columns:
        df[target] = (df[date].dt.dayofweek + 1) % 7
    return df

Wie im letzten Schritt definieren wir den `CustomTransformer`, um die Werte von `weekday` zu korrigieren.

In [ ]:
drop_rows_transformer = CustomTransformer(function=drop_rows, config={"target": "windspeed", "thresh": 0.0})

weekday_by_date = CustomTransformer(function=process_weekday, config={"target": "weekday", "date": "dteday"})

data_correction_pipeline = Pipeline([
    ("drop_rows", drop_rows_transformer),
    ("weekday_by_date", weekday_by_date)])

In [ ]:
pipeline = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("data_correction_pipeline", data_correction_pipeline),
    ("minimal_pipeline", minimal_pipeline)])

In [ ]:
result = pipeline.fit_transform(train_data[features])

In [ ]:
result_features = pipeline.named_steps['minimal_pipeline'].named_steps['imputer'].get_feature_names_out()

In [ ]:
filtered_indices = train_data[train_data['windspeed'] > 0.0].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_02 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_02.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_02.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Nachdem die Daten korregiert wurden, ist zu erkennen, dass einige Ausreißer mit den falschen Werten von `windspeed` entfernt wurden.

Zudem ist eine leichte Verbesserung in den Preformance Measures zu erkennen.

## 6 Outlier Handling

Es ist zu sehen, dass die Gesamtanzahl der geliehenen Fahrräder und die Anzahl der durch `casual` Nutzer geliehenen Fahrräder unrealistisch hohe Maximalwerte aufweisen.

In [ ]:
train_data[['casual', 'cnt']].describe().T

In [ ]:
zscores = stats.zscore(train_data["casual"])
thresh = 3.0
outliers = train_data[abs(zscores) > thresh]
outliers

Da `cnt` die Summe aus `casual` und `registered` ist, werden beim Entfernen der Outlier von `casual` auch die Outlier von `cnt` entfernt.

In [ ]:
def remove_outliers(df: pd.DataFrame, target: str, threshold: float = 3.0) -> pd.DataFrame:
    """
    Drop outliers from target column based on Z-scores.
    
    Args:
        df (pd.DataFrame): DataFrame to process.
        target (str): Target column name for outlier removal.
        threshold (float): Z-score threshold to consider a data point an outlier.
    
    Returns:
        pd.DataFrame: DataFrame with outliers removed from the specified target column.
    """
    if target in df.columns:
        # Calculate Z-scores for the target column
        z_scores = stats.zscore(df[target].dropna())
        
        # Identify outliers
        outliers = df[abs(z_scores) > threshold]
        
        df = df.drop(outliers.index, errors="ignore")

        print(f"{df.shape[0]} rows remaining after removing outliers.")
    return df

In [ ]:
remove_outliers_transformer = CustomTransformer(function=remove_outliers, config={"target": "casual"})

drop_feature_01 = CustomTransformer(function=drop_features, config={"columns": ["cnt", "casual", "registered"]})

pipeline = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("data_correction_pipeline", data_correction_pipeline),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_feature_01", drop_feature_01),
    ("minimal_pipeline", minimal_pipeline)])

In [ ]:
result = pipeline.fit_transform(train_data)

In [ ]:
result_features = pipeline.named_steps['minimal_pipeline'].named_steps['imputer'].get_feature_names_out()

In [ ]:
z_scores = abs(stats.zscore(train_data["casual"].dropna()))
filtered_indices = train_data[(z_scores < 3) & (train_data['windspeed'] > 0.0)].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_03 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_03.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_03.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Es ist zu erkennen, dass die Performace Measures, nach der Entfernung der Outliers, eine wesentliche Verbesserung zeigen.

## 7 Feature Transformation

### 7.1 One Hot Encoding

Für die Variablen `weekday`, `season` und `weathersit` erstellen wir mittels One Hot Encoding separate binäre Spalten für jede Ausprägung. 

Anschließend entfernen wir die ursprünglichen drei Spalten.

In [ ]:
column_transformer = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(sparse_output=False, drop="first", dtype=int), ['weekday', 'season', 'weathersit'])],
    remainder='passthrough'
)

In [ ]:
pipeline_01 = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("data_correction_pipeline", data_correction_pipeline),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_feature_01", drop_feature_01),
    ("numeric_values", numeric_values)])

In [ ]:
pipeline_02 = Pipeline([
    ("column_transformer", column_transformer),
    ("imputer", SimpleImputer(strategy="mean"))])

In [ ]:
pipeline = Pipeline([
    ("pipeline_01", pipeline_01),
    ("pipeline_02", pipeline_02)])

In [ ]:
result = pipeline.fit_transform(train_data)

In [ ]:
result_features = pipeline.named_steps["pipeline_02"].named_steps['column_transformer'].get_feature_names_out()

In [ ]:
z_scores = abs(stats.zscore(train_data["casual"].dropna()))
filtered_indices = train_data[(z_scores < 3) & (train_data['windspeed'] > 0.0)].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_04 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_04.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_04.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Nach dem Anwenden von One Hot Encoding zeigt sich erneut eine leicht Verbesserung der Performance Measures.

### 7.2 Normalisierung 

Im Folgenden werden alle Features mit Hilfe eines ```StandardScalers``` normalisiert, um den Einfluss der Gewichtung einzelner Features, aufgrund ihrer Wertebereiche, zu eliminiern.

Es ist zu erwarten, dass der Mean Absolute Error durch die Normalisierung geringer wird.

In [ ]:
column_transformer_02 = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(sparse_output=False, drop="first", dtype=int), ['weekday', 'season', 'weathersit'])],
    remainder=StandardScaler()
)

In [ ]:
pipeline_02 = Pipeline([
    ("column_transformer_02", column_transformer_02),
    ("imputer", SimpleImputer(strategy="mean"))])

In [ ]:
pipeline = Pipeline([
    ("pipeline_01", pipeline_01),
    ("pipeline_02", pipeline_02)])

In [ ]:
result = pipeline.fit_transform(train_data)

In [ ]:
z_scores = abs(stats.zscore(train_data["casual"].dropna()))
filtered_indices = train_data[(z_scores < 3) & (train_data['windspeed'] > 0.0)].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_05 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_05.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_05.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Durch das Anwenden des `StandardScaler` zeigt sich eine leichte Verbesserung des Mean Absolute Errors.

## 8 Multicolinearity

In der Korrelationsmatrix ist zu sehen, dass die Variablen `atemp`, und `temp` eine starke Korrelation aufweisen. 

Dies kann später zu Problemen bei der Regression führen, daher werden wir nur `atemp` behalten.

Wir verwenden die gefühlte Temperatur, da diese zu einem Grad aus der tatasächlichen Temperatur und der Luftfeuchte hervorgeht.

In [ ]:
drop_feature_02 = CustomTransformer(function=drop_features, config={"columns": ["temp"]})

In [ ]:
pipeline_01 = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("data_correction_pipeline", data_correction_pipeline),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_feature", drop_feature),
    ("drop_feature_01", drop_feature_01),
    ("drop_feature_02", drop_feature_02),
    ("numeric_values", numeric_values)])

In [ ]:
pipeline = Pipeline([
    ("pipeline_01", pipeline_01),
    ("pipeline_02", pipeline_02)])

In [ ]:
result = pipeline.fit_transform(train_data)

In [ ]:
result_features = pipeline.named_steps["pipeline_02"].named_steps['column_transformer_02'].get_feature_names_out()

In [ ]:
z_scores = abs(stats.zscore(train_data["casual"].dropna()))
filtered_indices = train_data[(z_scores < 3) & (train_data['windspeed'] > 0.0)].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_06 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
baseline_model_06.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_06.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Durch das Entfernen von `temp` sehen wir eine leichte Verschlechterung des Mean Absolute Errors.

Basierend auf der am Ende von Task 1 aufgestellen Vermutung, dass `hum` mit der gefühlten Temperatur `atemp` Colienar ist, schauen wir uns im Folgenden die Performance des Modell nochmal ohne `hum` an.

In [ ]:
drop_feature_03 = CustomTransformer(function=drop_features, config={"columns": ["hum"]})

In [ ]:
pipeline_01 = Pipeline([
    ("missing_value_pipeline", missing_value_pipeline),
    ("data_correction_pipeline", data_correction_pipeline),
    ("remove_outliers", remove_outliers_transformer),
    ("drop_feature", drop_feature),
    ("drop_feature_01", drop_feature_01),
    ("drop_feature_02", drop_feature_02),
    ("drop_feature_03", drop_feature_03),
    ("numeric_values", numeric_values)])

In [ ]:
pipeline = Pipeline([
    ("pipeline_01", pipeline_01),
    ("pipeline_02", pipeline_02)])

In [ ]:
result = pipeline.fit_transform(train_data)

In [ ]:
result_features = pipeline.named_steps["pipeline_02"].named_steps['column_transformer_02'].get_feature_names_out()

In [ ]:
z_scores = abs(stats.zscore(train_data["casual"].dropna()))
filtered_indices = train_data[(z_scores < 3) & (train_data['windspeed'] > 0.0)].index
result = pd.DataFrame(result, columns=result_features, index=filtered_indices)

In [ ]:
result.head().T

In [ ]:
result.describe().T

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(result, train_data.loc[filtered_indices]["cnt"], test_size=0.2, random_state=42)

In [ ]:
baseline_model_07 = Pipeline([("baseline_model", LinearRegression())])

In [ ]:
final_pipeline = baseline_model_07.fit(X_train, y_train)

In [ ]:
y_pred = baseline_model_07.predict(X_validate)

In [ ]:
_, _ = performance_measures(y_validate, y_pred)

In [ ]:
actual_vs_predicted(y_validate, y_pred)

Das Enfernen von `hum` führt zu einer leichten Verbesserung der Performance Measures.

Zuletzt exportieren wir das Datenset als CSV und speichern das Model.

In [ ]:
test_result = pipeline.fit_transform(test_data)
test_result_features = pipeline.named_steps["pipeline_02"].named_steps['column_transformer_02'].get_feature_names_out()
z_scores = abs(stats.zscore(test_data["casual"].dropna()))
test_filtered_indices = test_data[(z_scores < 3) & (test_data['windspeed'] > 0.0)].index
test_result = pd.DataFrame(test_result, columns=test_result_features, index=test_filtered_indices)
test_result["cnt"] = test_data.loc[test_filtered_indices]["cnt"]
test = test_result

In [ ]:
result["cnt"] = train_data.loc[filtered_indices]["cnt"]
train = result

In [ ]:
final_model = baseline_model_07["baseline_model"]
train.to_csv(f"../data/BikeRentalDaily_train_processed.csv", sep=";")
test.to_csv(f"../data/BikeRentalDaily_test_processed.csv", sep=";")

model_filename = '../models/final_model.pkl'
pickle.dump(final_model, open(model_filename, 'wb'))
model_features = '../models/final_model_features.pkl'
pickle.dump(result.columns, open(model_features, 'wb'))

Validierung des Modell mit den Testdaten.

In [ ]:
validation_data = pd.read_csv(f"../data/BikeRentalDaily_test_processed.csv", sep=";", index_col=0)

In [ ]:
validation_data.columns

In [ ]:
model = pickle.load(open(model_filename, 'rb'))

In [ ]:
features = pickle.load(open(model_features, 'rb'))

In [ ]:
features = validation_data.columns.drop(["cnt"])

In [ ]:
y_pred = model.predict(validation_data[features])

In [ ]:
_, _ = performance_measures(validation_data["cnt"], y_pred)

In [ ]:
actual_vs_predicted(validation_data["cnt"], y_pred)